In [ ]:
#!pip install Levenshtein
#!pip install sentence_transformers

In [1]:
import intellikit as ik
import pandas as pd

In [2]:
# Define your DataFrame on which the recommendation system is going to be based
df = pd.DataFrame({
    'feature1': ['apple', 'orange', 'banana', 'grape'],
    'feature2': ['red', 'green', 'blue', 'yellow'],
    'feature3': ['small', 'large', 'medium', 'small'],
    'feature4': [1, 2, 3, 4],
    'feature5': [10, 20, 30, 40]
})

#Define your query 
query = pd.DataFrame({
    'feature1': ['apple'],
    'feature2': ['yellow'],
    'feature3': ['small'],
    'feature4': [3],
    'feature5': [30]

})

#Define you similarity calculation methods for your project
hamming = ik.sim_hamming
levenshtein = ik.sim_levenshtein
level = ik.sim_level
abs_diff = ik.sim_difference

# Assign the appro*priate similarity calculation functions to each feature
similarity_functions = {
    'feature1': hamming,
    'feature2': levenshtein,
    'feature3': level,
    'feature4': abs_diff,
    'feature5': abs_diff
}

In [4]:
# Applying the methods and weights and retrieving the top results using the linear retriever
feature_weights = {
    'feature1': 0.4,
    'feature2': 0.2,
    'feature3': 0.1,
    'feature4': 0.1,
    'feature5': 0.2
}

top_n = 2  # Number of top similar results to return
top_similar_cases_linear = ik.linearRetriever(df, query, similarity_functions, feature_weights, top_n)
print("Top similar cases (Linear):")
print(top_similar_cases_linear)


Top similar cases (Linear):
  feature1 feature2 feature3  feature4  feature5
0    apple      red    small         1        10
3    grape   yellow    small         4        40
